In [1]:
import pandas as pd
import requests
from io import StringIO
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest
from sklearn.metrics import classification_report, roc_auc_score
import numpy as np

# URL του αρχείου CSV
url = 'https://raw.githubusercontent.com/kdemertzis/EKPA/main/Data/DarkNet.csv'

# Κατεβάζουμε το αρχείο CSV
response = requests.get(url)
data = pd.read_csv(StringIO(response.text))

# Επισκόπηση των δεδομένων
print(data.head())
print(data.info())

# Ελέγχουμε τις στήλες που περιέχουν μη αριθμητικά δεδομένα και τις μοναδικές τους τιμές
non_numeric_cols = data.select_dtypes(include=['object']).columns
for col in non_numeric_cols:
    unique_values = data[col].unique()
    print(f'{col}: {unique_values}')

# Αντικατάσταση μη αριθμητικών τιμών όπως '?'
for col in non_numeric_cols:
    data[col] = data[col].replace('?', 'Unknown')

# Έλεγχος αν υπάρχει η τιμή 'Unknown' και αντικατάστασή της με NaN
data.replace('Unknown', pd.NA, inplace=True)

# Διαγραφή γραμμών με NaN τιμές
data.dropna(inplace=True)

# Μετατροπή κατηγορηματικών δεδομένων σε αριθμητικά
data['Label-1'] = data['Label-1'].apply(lambda x: 1 if x == 'Tor' else 0)
data = pd.get_dummies(data, columns=['Label-2'], drop_first=True)

# Ελέγχουμε αν υπάρχουν άπειρες ή πολύ μεγάλες τιμές
if data.select_dtypes(include=[np.number]).apply(lambda x: np.any(np.isinf(x))).any() or np.any(data.values > np.finfo(np.float64).max):
    print("Υπάρχουν άπειρες ή πολύ μεγάλες τιμές στα δεδομένα.")
    # Αφαιρούμε τις γραμμές με άπειρες ή πολύ μεγάλες τιμές
    data.replace([np.inf, -np.inf], np.nan, inplace=True)
    data.dropna(inplace=True)



# Κανονικοποίηση των χαρακτηριστικών
scaler = StandardScaler()
data_scaled = scaler.fit_transform(data.drop(columns=['Src_IP', 'Dst_IP']))

# Εκπαίδευση μοντέλου Isolation Forest
model = IsolationForest(contamination=0.1, random_state=42)
model.fit(data_scaled)

# Πρόβλεψη ανωμαλιών
y_pred = model.predict(data_scaled)
y_pred = [1 if x == -1 else 0 for x in y_pred]  # -1 είναι ανωμαλία

# Αξιολόγηση μοντέλου
print(classification_report(data['Label-1'], y_pred))
print(f'ROC AUC: {roc_auc_score(data["Label-1"], y_pred)}')


<ipython-input-1-5ec2acb9f6ae>:14: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(StringIO(response.text))


          Src_IP  Src_Port          Dst_IP  Dst_Port  Protocol  Flow_Duration  \
0  10.152.152.11     57158   216.58.220.99       443         6            229   
1  10.152.152.11     57159   216.58.220.99       443         6            407   
2  10.152.152.11     57160   216.58.220.99       443         6            431   
3  10.152.152.11     49134  74.125.136.120       443         6            359   
4  10.152.152.11     34697  173.194.65.127     19305         6       10778451   

   Total_Fwd_Packet  Total_Bwd_packets  Total_Length_of_Fwd_Packet  \
0                 1                  1                           0   
1                 1                  1                           0   
2                 1                  1                           0   
3                 1                  1                           0   
4               591                400                       64530   

   Total_Length_of_Bwd_Packet  ...  Active_Mean  Active_Std  Active_Max  \
0                